In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix

In [5]:
data = pd.read_csv('../input/athlete-events/athlete_events.csv')

In [6]:
data.info()

In [7]:
data['Medal'] = data['Medal'].apply(lambda x: 1 if str(x) != 'nan' else 0)

In [8]:
data = data.drop(['ID','Name', 'Games'], axis=1)

In [9]:
data.groupby(['Medal', 'Sex']).mean()

In [10]:
for column in ['Age', 'Height', 'Weight']:
    data[column] = data.groupby(['Medal', 'Sex'])[column].apply(lambda x: x.fillna(x.mean()))

In [11]:
print("Total missing value:" , data.isna().sum().sum())

In [ ]:
#ENCODING

In [12]:
{column: len(data[column].unique()) for column in data.select_dtypes('object').columns}

In [13]:
def binary_encode(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [14]:
data = binary_encode(
    data,
    columns=['Sex', 'Season'],
    positive_values=['M', 'Summer']
)

data = onehot_encode(
    data,
    columns=['Team', 'NOC', 'City', 'Sport', 'Event'],
    prefixes=['T', 'N', 'C', 'S', 'E']
)

In [15]:
print("Remaining non-numeric columns:", len(data.select_dtypes('object').columns))

In [ ]:
#Visualizing Correlations

In [16]:
corr = data.loc[:, :'Medal'].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
plt.show()

In [ ]:
#Spliting/Scalling

In [17]:
y = data['Medal'].copy()
X = data.drop('Medal', axis=1).copy()

In [18]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

In [ ]:
#Training

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
model = KNeighborsClassifier(n_neighbors=5)

In [22]:
model.fit(X_train,y_train)

In [ ]:
acc = model.score(X_test, y_test)

In [ ]:
#Results

In [ ]:
print("Classification Report:\n\n", classification_report(y_test, Y_pred))

In [ ]:
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [ ]:
#END